In [ ]:
"""
Author: Wasif Rasool Qazi
Augumentation script for Sentinel-2-Dataset
"""


import os
import numpy as np
import rasterio
from PIL import Image
import albumentations as A

# Input directories
stackedImgDir = r"\Users\wasif\Desktop\multispectral\stacked"
maskDir = r"C:\Users\wasif\Desktop\multispectral\masks_resized"

# Output Directories
outputImgDir = r"C:\Users\wasif\Desktop\Augumented\images"
outputMaskDir = r"C:\Users\wasif\Desktop\Augumented\masks"
os.makedirs(outputImgDir, exist_ok=True)
os.makedirs(outputMaskDir, exist_ok=True)

# Use 5 Augumentation metric 
nAugs = 5

# Albumentations transformation
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.3),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=30, border_mode=0, p=0.4)
])

# Process all images
for file in os.listdir(stackedImgDir):
    # check if the file end with _stacked.jp2
    if file.endswith("_stacked.jp2"):
        baseName = file.replace("_stacked.jp2", "")
        stackedImgPath = os.path.join(stackedImgDir, file)
        # get masks corresponding to the stacked image 
        maskPath = os.path.join(maskDir, f"{baseName}_annotated_resized.png")

        if not os.path.exists(maskPath):
            print(f"Mask not found for {baseName}, skipping.")
            continue

        with rasterio.open(stackedImgPath) as src:
            imgStack = src.read()
            meta = src.meta

        imgStackNp = np.moveaxis(imgStack, 0, -1)
        maskNp = np.array(Image.open(maskPath).convert("L"))

        # Augumentation Loop
        for i in range(nAugs):
            augmented = transform(image=imgStackNp, mask=maskNp)
            augImg = np.moveaxis(augmented["image"], -1, 0)
            augMask = (augmented["mask"] > 127).astype(np.uint8) * 255

            # Check land and water pixels
            totalPixels = augMask.size
            landPixels = np.sum(augMask == 255)
            waterPixels = np.sum(augMask == 0)

            
            if landPixels / totalPixels >= 0.9:
                label = "land"
            elif waterPixels / totalPixels >= 0.9:
                label = "water"
            else:
                label = "mixed"

            # Output image and mask path , the names are kept same for easiness
            imgOutPath = os.path.join(outputImgDir, f"{baseName}_{label}_aug{i}.jp2")
            maskOutPath = os.path.join(outputMaskDir, f"{baseName}_{label}_aug{i}.png")

            # saving of aug masks and image
            try:
                with rasterio.open(
                    imgOutPath, "w",
                    driver="JP2OpenJPEG",
                    height=augImg.shape[1],
                    width=augImg.shape[2],
                    count=augImg.shape[0],
                    dtype=augImg.dtype,
                    crs=meta["crs"],
                    transform=meta["transform"]
                ) as dst:
                    dst.write(augImg)

                Image.fromarray(augMask).save(maskOutPath)

                print(f"Image: {imgOutPath} | Mask: {maskOutPath}")
            except Exception as e:
                print(f" Failed to save for {baseName}")

        print(f"Finished augmenting: {baseName}\n")
        print('Finished')


c:\Users\wasif\AppData\Local\Programs\Python\Python313\Lib\site-packages\albumentations\core\validation.py:87: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
c:\Users\wasif\AppData\Local\Programs\Python\Python313\Lib\site-packages\PIL\Image.py:3402: DecompressionBombWarning: Image size (120560400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Image: C:\Users\wasif\Desktop\Augumented\images\Image_10_mixed_aug0.jp2 | Mask: C:\Users\wasif\Desktop\Augumented\masks\Image_10_mixed_aug0.png
Image: C:\Users\wasif\Desktop\Augumented\images\Image_10_mixed_aug1.jp2 | Mask: C:\Users\wasif\Desktop\Augumented\masks\Image_10_mixed_aug1.png
Image: C:\Users\wasif\Desktop\Augumented\images\Image_10_mixed_aug2.jp2 | Mask: C:\Users\wasif\Desktop\Augumented\masks\Image_10_mixed_aug2.png
Image: C:\Users\wasif\Desktop\Augumented\images\Image_10_mixed_aug3.jp2 | Mask: C:\Users\wasif\Desktop\Augumented\masks\Image_10_mixed_aug3.png
Image: C:\Users\wasif\Desktop\Augumented\images\Image_10_mixed_aug4.jp2 | Mask: C:\Users\wasif\Desktop\Augumented\masks\Image_10_mixed_aug4.png
Finished augmenting: Image_10

Finished
Image: C:\Users\wasif\Desktop\Augumented\images\Image_14_mixed_aug0.jp2 | Mask: C:\Users\wasif\Desktop\Augumented\masks\Image_14_mixed_aug0.png
Image: C:\Users\wasif\Desktop\Augumented\images\Image_14_mixed_aug1.jp2 | Mask: C:\Users\wasif\D